# Języki i Biblioteki Analizy Danych
## Laboratorium 6.: Context managery
#### mgr inż. Zbigniew Kaleta

Problem:    

w programowaniu to:

In [ ]:
# set things up
# do something
# tear things down

przykład: 

In [ ]:
f = open("context_manager.ipynb")
f.readlines()
f.close()

jeśli wystąpi błąd pomiędy open a close to nie zostanie zamknięty plik 

Jest dobrze, dopóki operacje na obiekcie (np. pliku) nie rzucą wyjątku, bo wtedy `close()` się nie wykona.

Rozwiązanie 1.

In [1]:
# set things up
try:
    ... # do something
finally:
    ... # tear things down tutaj plik zostanie zamknięty 

tak poprawnie 

In [ ]:
f = open("context_manager.ipynb")
try:
    f.readlines()
finally:
    f.close()

Bezpiecznie, ale programiści lubią zapominać o `try-finally`, albo nie piszą ich z lenistwa, bo przecież "Co może pójść nie tak?"™

Rozwiązanie 2.

In [2]:
def controlled_execution(callback):
    # set things up
    try:
        callback(thing)
    finally:
        ... # tear things down

def my_function(thing):
    ... # do something with thing

controlled_execution(my_function) 

NameError: name 'thing' is not defined

In [3]:
def foo():
    print("X")
    raise IndexError()
    
def controlled_execution(f):
    print("Enter")
    try:
        f()
    finally:
        print("Exit")
    
controlled_execution(foo)

Enter
X
Exit


IndexError: 

Bezpiecznie, ale też niezbyt wygodnie, ze względu na konieczność definiowania funkcji, na potrzeby obsługi pliku (tutaj symuluje ją funkcja `foo`; zakładamy, że `controlled_execution` jest dostarczone).

Rozwiązanie 3.

In [4]:
def controlled_execution():
    # set things up
    try:
        yield thing
    finally:
        ... # tear things down

for thing in controlled_execution():
    ... # do something with thing

NameError: name 'thing' is not defined

In [5]:
def controlled_execution(filename):
    print("Enter")
    f = open(filename)
    try:
        yield f
    finally:
        f.close()
        print("Exit")
        
for x in controlled_execution("context_manager.ipynb"):
    print("X")
    raise KeyError()

Enter
X
Exit


KeyError: 

Bezpieczne i wygodne, ale `for` sugeruje wielokrotne wykonanie tego kodu.

Rozwiązanie 4.

In [6]:
class controlled_execution:
    def __enter__(self):
        # set things up
        return thing

    def __exit__(self, type, value, traceback):
        ... # tear things down

with controlled_execution() as thing: # blok specjalnie na to woprowadzony 
     ... # do something with thing

NameError: name 'thing' is not defined

O to chodziło! Osobny element składni. DEF context menegera

Po zakończeniu bloku `with` zasób będzie zwolniony.

przykład

In [7]:
with open("context_manager.ipynb") as notebook:
    print(notebook.read(50))

{
 "cells": [
  {
   "cell_type": "markdown",
   "


Taki zapis wymaga tylko minimalnie więcej wysiłku niż napisanie `notebook = open(...)` i mniej wysiłku niż pamiętanie jeszcze o `notebook.close()`.

In [8]:
notebook

<_io.TextIOWrapper name='context_manager.ipynb' mode='r' encoding='cp1252'>

plik będzie zamknięty wiec nie da się wykonać read()

In [9]:
notebook.read(50)

ValueError: I/O operation on closed file.

Przykładowy context manager:

In [10]:
class MyContextManager:
    
    def __enter__(self):
        print("Context prepared")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)

        
with MyContextManager() as f:
    print("Hello", f)

Context prepared
Hello <__main__.MyContextManager object at 0x0000021C55762FE0>
Context closed  None None None


In [ ]:
class MyContextManager:
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)
        return True  # zwrócenie True oznacza, że przyznajemy się do wyjątku, który został przekazany do exit i należy go uważać za obsłużony

        
with MyContextManager():
    raise Exception()  # <- never do that at home

### Context manager z użyciem dekoratora

In [11]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name,)
    try:
        yield
    finally:
        print("</%s>" % name,)

with tag("div"):
    with tag("a"):
        print("foo",)

<div>
<a>
foo
</a>
</div>


In [12]:
with tag("h1"):
    1 + '1'
    
print()

<h1>
</h1>


TypeError: unsupported operand type(s) for +: 'int' and 'str'

Przykład z tymczasowym folderem

(za http://stackoverflow.com/questions/3012488/what-is-the-python-with-statement-designed-for)

In [13]:
from tempfile import mkdtemp
from shutil import rmtree

@contextmanager
def temporary_dir(*args, **kwds):
    name = mkdtemp(*args, **kwds)
    try:
        yield name
    finally:
        rmtree(name)

with temporary_dir(".temp") as dirname:
    print("doing sth with", dirname)

doing sth with C:\Users\Radosław\AppData\Local\Temp\tmpukyfthxf.temp
